In [4]:
# import logging
%load_ext autoreload
%autoreload 2

import sys
import os
from collections import Counter
import dotenv
import pandas as pd

sys.path.append('../') # allow importing of other modules
import rag_gold.rag_pipeline as rpl

# The following loads file with environment variables (such as LLM API Keys)
# change path to file as needed
dotenv.load_dotenv(os.environ['HOME'] + '/profile.env', override=True)

from llama_index.core import SimpleDirectoryReader
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
documents = SimpleDirectoryReader("../rag_docs").load_data()


Counter({('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.preprocessed2.translated.txt', 'text/plain'): 1, ('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.preprocessed2.txt', 'text/plain'): 1, ('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.translated.txt', 'text/plain'): 1, ('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/reglamento-maestria-web-2024.preprocessed2.txt', 'text/plain'): 1})


In [ ]:


chunk_size=300
chunk_overlap=250

embed_model = rpl.HuggingFaceEmbedding(rpl.DEFAULT_EMB_MODEL)

vecstore_idx = rpl.make_vectore_index(embedding=embed_model,
                                      documents=documents,
                                      chunk_size=chunk_size,
                                      chunk_overlap=chunk_overlap
                                      )


In [103]:
len(vecstore_idx.docstore.docs)

1541

In [ ]:
r = vecstore_idx.as_retriever()
dir(r)
r.similarity_top_k = 3

# r.retrieve("¿Cuál es la naturaleza jurídica de la Universidad de los Andes según sus estatutos?")

query_str = test_df.iloc[1].Pregunta
print(query_str)

query_embedding = embed_model.get_query_embedding(query_str)

vector_store_query = rpl.VectorStoreQuery(
        query_embedding=query_embedding,
        similarity_top_k=3,
        mode="default",
)

query_result = vecstore_idx.vector_store.query(vector_store_query)

id_2_node = vecstore_idx.docstore.docs

# nodes_with_scores = []
for idx, node_id in enumerate(query_result.ids):
    node = id_2_node[node_id]
    score = query_result.similarities[idx]
    print(f"\n==== idx: {idx}  node_id: {node_id}====\n{node.get_text()}\n")
    # nodes_with_scores.append(NodeWithScore(node=node, score=score))



¿Dónde se encuentra el domicilio principal de la Universidad de los Andes?

==== idx: 0  node_id: 23c3b322-699d-4113-81a8-19150870ff81====
## VISIÓN

En el año 2020, la Universidad de los Andes será una institución líder y referente en educación superior en América Latina, por la excelencia, pertinencia y relevancia de sus programas académicos, su calidad docente y la investigación que desarrolla.

## VALORES

La Universidad profesa y acoge los siguientes valores:
Excelencia: Compromiso con el desarrollo de la máximas capacidades y posibilidades para aportar a la sociedad.


==== idx: 1  node_id: 4cd39ccb-799e-4ed1-9fc4-4180a46f71b7====
## PREÁMBULO

La Universidad de los Andes se fundó como institución de enseñanza superior, de carácter privado, sin ánimo de lucro, para ofrecer a la juventud colombiana nuevas oportunidades de educación, en un ambiente de respeto por la pluralidad de ideas y la diversidad de pensamiento.
La Universidad se considera una comunidad compuesta por sus funda

In [33]:
import pandas as pd
test_df = pd.read_csv("../data/test_uniandes.csv")

test_df.iloc[1]

def format_question(row: pd.Series) -> str:
    return """{question}
    Selecciona una sola de las siguientes opciones:
    a. {op1}
    b. {op2}
    c. {op3}
    d. {op4}
    IMPORTANTE: Responde solo con una letra a, b, c, o d, correspondiente a la mejor respuesta. No agregues comentarios ni justificaciones.
    """.format(
        question=row['Pregunta'],
        op1=row['Opcion1'],
        op2=row['Opcion2'],
        op3=row['Opcion3'],
        op4=row['Opcion4'],
    )

print(format_question(test_df.iloc[0]))

¿Cuál es la naturaleza jurídica de la Universidad de los Andes según sus estatutos?
    Selecciona una sola de las siguientes opciones:
    a. Entidad privada con ánimo de lucro
    b. Institución pública
    c. Corporación de utilidad común sin ánimo de lucro
    d. Fundación educativa mixta
    IMPORTANTE: Responde solo con una letra a, b, c, o d, correspondiente a la mejor respuesta. No agregues comentarios ni justificaciones.
    


In [106]:
from src.utils import get_secret
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(
        model="claude-3-5-sonnet-20241022",
        api_key=get_secret("ANTHROPIC_API_KEY")
    )

chunk_size=300
chunk_overlap=250
top_k = 3

q_engine = rpl.make_query_engine(documents,
                                 chunk_size=chunk_size,
                                 chunk_overlap=chunk_overlap,
                                 llm=llm,
                                 vs_type="simple",
                                 similarity_top_k=top_k)

q_engine = RetrieverQueryEngine.from_args(q_engine.retriever, llm=llm)

Returning secret from environment variable `ANTHROPIC_API_KEY`=`sk...AA`
Using LLM=callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f894722d050> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x7f8a2b492a20> completion_to_prompt=<function default_completion_to_prompt at 0x7f8a2b3b42c0> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None model='claude-3-5-sonnet-20241022' temperature=0.1 max_tokens=512 base_url=None timeout=None max_retries=10 additional_kwargs={}


Generating embeddings: 0it [00:00, ?it/s]

In [107]:
retriever = q_engine.retriever

row = test_df.iloc[1]
fmt_question = format_question(row)
print(fmt_question)
raw_question=row['Pregunta']
retriever._debug_query(raw_question)

response = q_engine.query(fmt_question)
response_str = str(response)

¿Dónde se encuentra el domicilio principal de la Universidad de los Andes?
    Selecciona una sola de las siguientes opciones:
    a. En la ciudad de Cali
    b. En la ciudad de Medellín
    c. En la ciudad de Bogotá
    d. En cualquier lugar de Colombia
    IMPORTANTE: Responde solo con una letra a, b, c, o d, correspondiente a la mejor respuesta. No agregues comentarios ni justificaciones.
    

==== idx: 0  node_id: d6c1b357-e045-4435-a14c-f8c79aa5f954  SCORE: 0.7178====
## VISIÓN

En el año 2020, la Universidad de los Andes será una institución líder y referente en educación superior en América Latina, por la excelencia, pertinencia y relevancia de sus programas académicos, su calidad docente y la investigación que desarrolla.

## VALORES

La Universidad profesa y acoge los siguientes valores:
Excelencia: Compromiso con el desarrollo de la máximas capacidades y posibilidades para aportar a la sociedad.
Integridad: Rectitud de las actuaciones conforme a principios relativos a la dig

In [108]:
response_str

'No puedo determinar la respuesta a esta pregunta ya que la información sobre el domicilio principal de la Universidad de los Andes no está incluida en el contexto proporcionado.'

In [109]:
answers_by_id = {}

In [110]:
# Responde solo con un número entre 1 y 4, correspondiente a la respuesta.
import time

q_engine.retriever._verbose = False

for _, row in test_df.iterrows():
    id_ = row['ID']
    resp = answers_by_id.get(id_)
    if  resp in {'a', 'b', 'c', 'd'}:
        print(f"\nID={id_} already answered, resp: {resp}")
        continue

    formatted_question = format_question(row)
    # print(id_, formatted_question)
    response = q_engine.query(formatted_question)
    response_str = str(response)
    time.sleep(1)
    resp_l0 = response_str.split('\n')[0]
    print(f"\n{id_} - Pregunta: {row['Pregunta']} Respuesta: {resp_l0}\n\n")
    answers_by_id[id_] = response_str





1 - Pregunta: ¿Cuál es la naturaleza jurídica de la Universidad de los Andes según sus estatutos? Respuesta: c



2 - Pregunta: ¿Dónde se encuentra el domicilio principal de la Universidad de los Andes? Respuesta: No puedo determinar la respuesta correcta ya que el contexto proporcionado no menciona específicamente la ubicación o domicilio principal de la Universidad de los Andes.



3 - Pregunta: ¿Qué órgano de gobierno tiene la responsabilidad de fijar la política general de la Universidad? Respuesta: c



4 - Pregunta: ¿Cuál es la duración del período inicial para el rector de la Universidad? Respuesta: No puedo determinar la respuesta correcta ya que la información sobre la duración del período del rector no está presente en el contexto proporcionado. El contexto contiene información sobre plazos para obtener el grado de maestría, grados póstumos y entrega de calificaciones, pero nada relacionado con el período del rector.



5 - Pregunta: ¿Cuántos consejeros ordinarios puede tene

In [112]:
answer_count = sum(1 if v in "abcd" else 0 for v in answers_by_id.values())
print("ANSWER COUNT: ", answer_count)

answers_by_id

ANSWER COUNT:  58


{1: 'c',
 2: 'No puedo determinar la respuesta correcta ya que el contexto proporcionado no menciona específicamente la ubicación o domicilio principal de la Universidad de los Andes.',
 3: 'c',
 4: 'No puedo determinar la respuesta correcta ya que la información sobre la duración del período del rector no está presente en el contexto proporcionado. El contexto contiene información sobre plazos para obtener el grado de maestría, grados póstumos y entrega de calificaciones, pero nada relacionado con el período del rector.',
 5: 'The provided context does not contain information about the maximum number of regular council members (consejeros ordinarios) that the Superior Council can have. Therefore, I cannot determine which of the given options (a, b, c, or d) is correct.',
 6: 'No puedo determinar la respuesta correcta ya que el contexto proporcionado no contiene información específica sobre las funciones principales del Comité Directivo. Si bien el Comité Directivo es mencionado brevem

In [113]:
df = pd.Series(answers_by_id).reset_index()
df.columns= ['ID', f'Claude-3.5-ch{chunk_size}-k{top_k}']
f_path_out = f"../data/rag-answers-claude-3.5-ch{chunk_size}-ol{chunk_overlap}-k{top_k}-DONT-SUBMIT.csv"
print(f_path_out)
df.to_csv(f_path_out, index=False)
df

../data/rag-answers-claude-3.5-ch300-ol250-k3-DONT-SUBMIT.csv


,ID,Claude-3.5-ch300-k3
0,1,c
1,2,No puedo determinar la respuesta correcta ya q...
2,3,c
3,4,No puedo determinar la respuesta correcta ya q...
4,5,The provided context does not contain informat...
...,...,...
95,96,d
96,97,La información proporcionada en el contexto so...
97,98,b
98,99,c


In [7]:
# Clean up  model answers

df = pd.read_csv("../data/rag-answers-claude-ch700-2.1-DONT-SUBMIT.csv")
df

,ID,Claude-2.1-Answer-ch700
0,1,Unfortunately I do not have enough context in ...
1,2,c. En la ciudad de Bogotá
2,3,c
3,4,No hay información en el contexto proporcionad...
4,5,No hay suficiente información en el contexto p...
...,...,...
95,96,d. 3.0\n\nLa respuesta correcta es d. 3.0\n\nS...
96,97,b) 35%
97,98,b. Título de Especialista
98,99,c


In [15]:
import re
def extract_letter(ans: str, verbose=False) -> str:
    m = re.match(r"([a-dA-D])(\)|\.)*", ans)
    if m:
        return m.group(1)
    else:
        if verbose:
            print("???", ans)
        return "?"

ex = df.iloc[:, 1].apply(extract_letter)

In [17]:
(ex != '?').sum()

76

In [134]:
ans_vers = {
    "groq288": "../data/rag-answers-groq-ch288-DONT-SUBMIT.csv",
    "claude2.1-ch700": "../data/rag-answers-claude-ch700-2.1-DONT-SUBMIT.csv",
    "claude3.5-ch100-k5": "../data/rag-answers-claude-3.5-ch100-ol80-k5-DONT-SUBMIT.csv",
    "claude3.5-ch300-k3": "../data/rag-answers-claude-3.5-ch300-ol250-k3-DONT-SUBMIT.csv"
}

def combine_answers_models(ans_vers):

    ret_df = None
    for model_key, file_path in ans_vers.items():
        df = pd.read_csv(file_path)
        assert df.columns[0] == 'ID'
        df.columns = ['ID', model_key]
        df[model_key] = df[model_key].apply(extract_letter)
        if ret_df is None:
            ret_df = df
        else:
            ret_df = pd.merge(ret_df, df, on='ID')

        del df

    ans_cols = [col for col in ret_df.columns if col != 'ID']
    print(ans_cols)

    def all_ans(row) -> str:
        # print(row)
        anss = [ row[col] for col in ans_cols ]
        if len(set(anss)) == 1:
            return anss[0]
        else:
            return '?'


    def cnts(row) -> str:
        return dict(Counter(row[ans_cols]))

    def ans_agree_2(row) -> str:
        """The ones that answered agree answer and they agree rest don't know"""
        # print(row)

        d = row['cnts'].copy()
        if '?' in d:
            del d['?']

        if len(d) == 1 and next(iter(d.values())) >= 2:
            return next(iter(d.keys()))
        else:
            return "?"


    def ans_agree_1(row) -> str:
        """The ones that answered agree answer and they agree rest don't know"""
        # print(row)

        d = row['cnts'].copy()
        if '?' in d:
            del d['?']

        if len(d) == 1:
            return next(iter(d.keys()))
        else:
            return "?"


    ret_df['cnts'] = ret_df.apply(cnts, axis=1)
    ret_df['all'] = ret_df.apply(all_ans, axis=1)
    ret_df['ans_agree_1'] = ret_df.apply(ans_agree_1, axis=1)
    ret_df['ans_agree_2'] = ret_df.apply(ans_agree_2, axis=1)

    return ret_df


comb = combine_answers_models(ans_vers)
print("ALL ANSWER and AGREE: ", (comb['all'] != '?').sum())
print("ANSWER and AGREE 2 : ", (comb['ans_agree_2'] != '?').sum())
print("ANSWER and AGREE 1 : ", (comb['ans_agree_1'] != '?').sum())
comb

comb.to_csv("../data/rag_gold.comb.DO-NOT-SUBMIT.csv", index=False)

test_x_df = test_df.merge(comb, on="ID")
test_x_df.to_csv("../data/test_uniandes_w_ans.csv", index=False)
test_x_df

['groq288', 'claude2.1-ch700', 'claude3.5-ch100-k5', 'claude3.5-ch300-k3']
ALL ANSWER and AGREE:  21
ANSWER and AGREE 2 :  58
ANSWER and AGREE 1 :  70


,ID,Pregunta,Opcion1,Opcion2,Opcion3,Opcion4,groq288,claude2.1-ch700,claude3.5-ch100-k5,claude3.5-ch300-k3,cnts,all,ans_agree_1,ans_agree_2
0,1,¿Cuál es la naturaleza jurídica de la Universi...,Entidad privada con ánimo de lucro,Institución pública,Corporación de utilidad común sin ánimo de lucro,Fundación educativa mixta,c,?,?,c,"{'c': 2, '?': 2}",?,c,c
1,2,¿Dónde se encuentra el domicilio principal de ...,En la ciudad de Cali,En la ciudad de Medellín,En la ciudad de Bogotá,En cualquier lugar de Colombia,?,c,c,?,"{'?': 2, 'c': 2}",?,c,c
2,3,¿Qué órgano de gobierno tiene la responsabilid...,El Comité Directivo,El Consejo Académico,El Consejo Superior,El Rectorado,c,c,c,c,{'c': 4},c,c,c
3,4,¿Cuál es la duración del período inicial para ...,Dos años,Tres años,Cuatro años,Cinco años,?,?,a,?,"{'?': 3, 'a': 1}",?,a,?
4,5,¿Cuántos consejeros ordinarios puede tener com...,20,30,45,50,?,?,?,?,{'?': 4},?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,¿Cuál es la calificación mínima en la escala n...,1.0,1.5,2.0,3.0,b,d,b,d,"{'b': 2, 'd': 2}",?,?,?
96,97,¿Qué porcentaje máximo puede asignarse a una s...,40%,35%,50%,30%,b,b,b,?,"{'b': 3, '?': 1}",?,b,b
97,98,¿Qué título puede solicitar un estudiante de m...,Título de Doctorado,Título de Especialista,Certificado de aprobación,Diploma de asistencia,b,b,b,b,{'b': 4},b,b,b
98,99,¿Qué entidad está a cargo de la aprobación de ...,Consejo Superior,Comité de Asuntos Estudiantiles,El Decano de la facultad correspondiente,Rector,c,c,c,c,{'c': 4},c,c,c


,ID,Pregunta,Opcion1,Opcion2,Opcion3,Opcion4,groq288,claude2.1-ch700,claude3.5-ch100-k5,claude3.5-ch300-k3,cnts,all,ans_agree_1,ans_agree_2
0,1,¿Cuál es la naturaleza jurídica de la Universi...,Entidad privada con ánimo de lucro,Institución pública,Corporación de utilidad común sin ánimo de lucro,Fundación educativa mixta,c,?,?,c,"{'c': 2, '?': 2}",?,2,2
1,2,¿Dónde se encuentra el domicilio principal de ...,En la ciudad de Cali,En la ciudad de Medellín,En la ciudad de Bogotá,En cualquier lugar de Colombia,?,c,c,?,"{'?': 2, 'c': 2}",?,2,2
2,3,¿Qué órgano de gobierno tiene la responsabilid...,El Comité Directivo,El Consejo Académico,El Consejo Superior,El Rectorado,c,c,c,c,{'c': 4},c,4,4
3,4,¿Cuál es la duración del período inicial para ...,Dos años,Tres años,Cuatro años,Cinco años,?,?,a,?,"{'?': 3, 'a': 1}",?,1,?
4,5,¿Cuántos consejeros ordinarios puede tener com...,20,30,45,50,?,?,?,?,{'?': 4},?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,¿Cuál es la calificación mínima en la escala n...,1.0,1.5,2.0,3.0,b,d,b,d,"{'b': 2, 'd': 2}",?,?,?
96,97,¿Qué porcentaje máximo puede asignarse a una s...,40%,35%,50%,30%,b,b,b,?,"{'b': 3, '?': 1}",?,3,3
97,98,¿Qué título puede solicitar un estudiante de m...,Título de Doctorado,Título de Especialista,Certificado de aprobación,Diploma de asistencia,b,b,b,b,{'b': 4},b,4,4
98,99,¿Qué entidad está a cargo de la aprobación de ...,Consejo Superior,Comité de Asuntos Estudiantiles,El Decano de la facultad correspondiente,Rector,c,c,c,c,{'c': 4},c,4,4


In [ ]:
# preds_df = pd.read_csv("../data/r16-e124-bs1024-s64-spa-v1.csv")


In [ ]:
def accuracy_metrics(fpath: str) -> dict[str, float]:
    preds_df = pd.read_csv(fpath)

    test_x_df = pd.read_csv("../data/test_uniandes_w_ans.csv")
    ref_ans_cols = ["all", "ans_agree_1", "ans_agree_2"]

    # map a -> 1, b -> 2, c -> 3, d -> 4
    let_to_idx = dict(zip("abcd", [1, 2, 3, 4]))

    for col in ref_ans_cols:
        test_x_df[col] = test_x_df[col].map(lambda let: let_to_idx.get(let))

    # print(test_x_df.head())

    preds_df = preds_df.copy()
    assert len(preds_df.columns) == 2
    assert preds_df.columns[0] == "ID"
    preds_df.columns = ["ID", "Respuesta"]
    assert preds_df.shape[0] == 100
    assert set(preds_df["ID"]) == set(range(1, 101))

    grading_df = test_x_df[['ID'] + ref_ans_cols].merge(preds_df, on="ID")

    for col in ref_ans_cols:
        ref_ans = test_x_df[col]
        # print(col, ref_ans.isnull().sum())
        has_ans = ref_ans.notna()
        # print(list(has_ans))
        correct = grading_df['Respuesta'][has_ans] == grading_df[col][has_ans]
        n_correct = correct.sum()
        n_total = has_ans.sum()
        print(f"{col:15s}: {n_correct:2d}/{n_total:2d} = {n_correct / n_total:.4f}")


accuracy_metrics(preds_df)


all            :  9/21 = 0.4286
ans_agree_1    : 21/70 = 0.3000
ans_agree_2    : 19/58 = 0.3276


In [156]:
grading_df['Respuesta'][has_ans]

0     3
1     3
2     2
5     4
6     1
7     3
9     3
11    4
16    1
17    2
23    4
25    3
29    4
30    3
31    1
32    4
33    4
35    2
36    4
37    3
38    4
39    1
40    2
41    3
42    4
44    4
45    4
46    4
47    1
48    4
49    4
50    4
53    3
54    3
59    4
60    3
61    4
63    4
68    4
72    4
74    3
75    4
77    3
78    3
80    3
84    3
85    3
86    4
87    3
88    4
91    3
92    3
93    3
94    1
96    4
97    3
98    3
99    2
Name: Respuesta, dtype: int64